In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
#import cubic_spline_planner

N_IND_SEARCH = 10  # Search index number
show_animation = True

In [2]:
!rm -f Data_Processor.pyc
Remove=True
if Remove==True:
    !rm -r Fig_iLQR
    !rm -r Rec_iLQR
    !rm -r Rec_iLQR_Raw

rm: cannot remove 'Fig_iLQR': No such file or directory
rm: cannot remove 'Rec_iLQR': No such file or directory
rm: cannot remove 'Rec_iLQR_Raw': No such file or directory


In [3]:
import numpy as np
import math
from copy import copy, deepcopy
TARGET_SPEED = 20# / 3.6  # [m/s] target speed
MAX_TIME=1500
T=5
NX=3
NU=2
DT=0.2
WB=50
MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 25#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -25#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 5.0  # maximum accel [m/ss]
    


In [4]:
from Motion_DFD import Motion,State
R = np.diag([1.0, 10.0])  # input cost matrix
Q = np.diag([1.0, 1.0,0.5])  # state cost matrix


def np_quad_form(a,b):
        res=a @ b @ a.T
        return res
    
    
    
def sim_n_steps(_x0,U,xref,N):
    _U=U.copy()
    X=np.zeros((NX,N+1),dtype=np.float64)
    X[:,0]=_x0
    
    total_grad=0
    loss=0
    for t in range(N):
        x_t=X[:,t]
        u_t=_U[:,t]
        xref_t=xref[:,t]
        x_t1=sim_1_step(x_t,u_t)
        loss += np_quad_form(u_t, R) + np_quad_form(xref_t - x_t, Q)
        #cc=calc_cost(x_t1,u_t,xref_t,t)
        X[:,t+1]=x_t1.copy()
    return X,loss,U

def sim_1_step(_X,_U):
    
    _state=State(x=_X[0],y=_X[1],v=_U[0],yaw=_X[2])
    nst,Const=Motion.get_next_state(_state,_U[0],_U[1])
    x_t1=np.array([nst.x,nst.y,nst.yaw],dtype=np.float64)
    return x_t1#np.array(x_t1,dtype=np.float64)

In [5]:
class iLQRController():
    def __init__(self):
        self.STATE_SIZE=NX
        self.INPUT_SIZE=NU
        self.max_iter=10
        self.dt=0.2
        self.lamb_factor=1.0
        self.lamb_max=10e4
        self.tN=T
        self.eps_converge=0.001
        
    def init_forward_matrix(self):
        self.f_x = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # df / dx
        self.f_u = np.zeros((self.tN, self.STATE_SIZE, self.INPUT_SIZE)) # df / du
        
    def init_cost(self):
        self.l = np.zeros((self.tN,1)) # immediate state cost 
        self.l_x = np.zeros((self.tN, self.STATE_SIZE)) # dl / dx
        self.l_xx = np.zeros((self.tN, self.STATE_SIZE, self.STATE_SIZE)) # d^2 l / dx^2
        self.l_u = np.zeros((self.tN, self.INPUT_SIZE)) # dl / du
        self.l_uu = np.zeros((self.tN, self.INPUT_SIZE, self.INPUT_SIZE)) # d^2 l / du^2
        self.l_ux = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # d^2 l / du / dx
    
    def calc_cost(self, _x, _u, _xref,t):
        
        
        self.l[t] = np_quad_form(_u, R) + np_quad_form(_xref - _x, Q)
        
        loss_x1=2*Q[0,0]*(_xref[0]-_x[0])*(-1)
        loss_x2=2*Q[1,1]*(_xref[1]-_x[1])*(-1)
        #loss_x3=2*Q[2,2]*(_xref[2]-_x[2])*(-1)
        loss_x3=2*Q[2,2]*(_xref[2]-_x[2])*(-1)
            
        self.l_x[t]=np.array([loss_x1,loss_x2,loss_x3])
        self.l_xx[t]=2 * Q
        
        Qdiag=np.array([Q[0,0], Q[1,1], Q[2,2]],dtype=np.float64)
        error=np.array(_xref-_x)
        
        loss_u1 = 2. * _u[0] * R[0,0] #+ np.sum(2*Qdiag*error*(-1)*_B[:,0])
        loss_u2 = 2. * _u[1] * R[1,1] #+ np.sum(2*Qdiag*error*(-1)*_B[:,1])
        
        self.l_u[t] = np.array([loss_u1, loss_u2])
        self.l_uu[t] = 2. * R
        self.l_ux[t] = np.zeros((NU, NX))
    
    def init_ks(self):
        self.k = np.zeros((self.tN, self.INPUT_SIZE)) # feedforward modification
        self.K = np.zeros((self.tN, self.INPUT_SIZE, self.STATE_SIZE)) # feedback gain

  
    def calc_forward_matrix(self,_X,_U,t):
        A, B, C = Motion.get_linear_model_matrix(_U[0],_X[2], None)
        self.f_x[t] = A
        self.f_u[t] = B 
        
    def ilqr(self, x0, U,xref): 
        
        lamb = 1.0 # regularization parameter
        sim_new_trajectory = True
        tN = U.shape[0] # number of time steps
        X=np.zeros((NX,T+1),dtype=np.float64)
        X[:,0]=x0.copy()
        for ii in range(self.max_iter):

            if sim_new_trajectory == True: 
                #X, cost, _ = sim_n_steps(x0, U,xref,T)
                #oldcost = copy(cost) # copy for exit condition check

                
                self.init_forward_matrix()
                self.init_cost()
                for t in range(self.tN):
                    x_t=X[:,t].copy()
                    u_t=U[:,t].copy()
                    xref_t=xref[:,t].copy()
                    self.calc_forward_matrix(x_t,u_t,t)
                    self.calc_cost(x_t, u_t,xref_t,t)
                    X[:,t+1]=sim_1_step(x_t,u_t)
                _cost =np.sum(self.l)
                sim_new_trajectory = False

            V = self.l[-1].copy()  
            V_x = self.l_x[-1].copy() 
            V_xx = self.l_xx[-1].copy() 
            self.init_ks()
 
            for t in range(self.tN-2, -1, -1):
                
                Q_x = self.l_x[t] + np.dot(self.f_x[t].T, V_x) 
                Q_u = self.l_u[t] + np.dot(self.f_u[t].T, V_x)

                Q_xx = self.l_xx[t] + np.dot(self.f_x[t].T, np.dot(V_xx, self.f_x[t])) 
                Q_ux = self.l_ux[t] + np.dot(self.f_u[t].T, np.dot(V_xx, self.f_x[t]))
                Q_uu = self.l_uu[t] + np.dot(self.f_u[t].T, np.dot(V_xx, self.f_u[t]))

                Q_uu_evals, Q_uu_evecs = np.linalg.eig(Q_uu)
                Q_uu_evals[Q_uu_evals < 0] = 0.0
                #Q_uu_evals += lamb
                Q_uu_inv = np.dot(Q_uu_evecs, np.dot(np.diag(1.0/Q_uu_evals), Q_uu_evecs.T))
                #k[t]=-1*l_u[t]
                self.k[t] = -1. * np.dot(Q_uu_inv, Q_u)
                self.K[t] = -1. * np.dot(Q_uu_inv, Q_ux)

                V_x = Q_x - np.dot(self.K[t].T, np.dot(Q_uu, self.k[t]))
                V_xx = Q_xx - np.dot(self.K[t].T, np.dot(Q_uu, self.K[t]))

            U_new = np.zeros((NU,T),dtype=np.float64)
            x_new = np.zeros((NX,T),dtype=np.float64)
            x_new = x0.copy()
            for t in range(self.tN - 1): 
               
                U_new[:,t] = U[:,t] + self.k[t] + np.dot(self.K[t], x_new - X[:,t])
                x_new_1= sim_1_step(x_new, U_new[:,t])
                x_new=x_new_1.copy()
            # evaluate the new trajectory 
            X_new, _new_cost , _= sim_n_steps(x0, U_new,xref,T)
            #if 1==1:
      
            if _new_cost < _cost: 
                # decrease lambda --> get closer to Newton's method
                lamb /= self.lamb_factor

                X = copy(X_new) 
                U = copy(U_new) 
                #oldcost = copy(cost)
                #cost = copy(cost_new)

                sim_new_trajectory = True 

                #print('...',_cost,_new_cost)
                if ii > 0 and ((abs(_cost-_new_cost)/_new_cost) < self.eps_converge):
                    print("break 1",ii)
                    break

            else: 
                # increase lambda --> closer to gradient descent
                lamb *= self.lamb_factor
                if lamb > self.lamb_max: 
                    print("break 2",ii)
                    break
        print('return ii',ii,_new_cost,_cost)
        return X, U, _cost
    

In [6]:
from Car_Env_Basic import Car_Env
from Data_Processor import Data_Processor
from Car_Plot import Move_Contour,Draw_Car

In [7]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
rec=False
max_len=350
qlen=25
car_move=True

Env=Car_Env(rec=rec,max_len=max_len,qlen=qlen,rec_path="Rec_iLQR")
[init_state,map_points]=Env.reset()
mpx=np.array(map_points[:max_len,0])#-ofx
mpy=np.array(map_points[:max_len,1])#-ofy
ix,iy,iyaw,ivel=init_state[0]
_traj=init_state[1][0]
_dind=init_state[1][1]
_end=init_state[1][2]
ofx,ofy,ofang=init_state[2]
img=init_state[3]
state = State(x=ix, y=iy, yaw=iyaw, v=ivel)
#print(init_state)

In [8]:
if rec==True:
    import os
    if not os.path.exists('Rec_iLQR/'):
        os.mkdir('Rec_iLQR')
    if not os.path.exists('Rec_iLQR_Raw/'):
        os.mkdir('Rec_iLQR_Raw')
    if not os.path.exists('Fig_iLQR/'):
        os.mkdir('Fig_iLQR')


In [9]:
import cv2
import matplotlib.gridspec as gridspec
import time
phi=0
rot_mat=np.array([[math.cos(phi),math.sin(phi)],[-math.sin(phi),math.cos(phi)]])
dis_time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]

log_state=[]
log_traj=[]
log_img=[]
log_act=[]
log_next_state=[]
log_reward=[]


#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
_index=0
#cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''


_Data_Processor=Data_Processor(_speed=TARGET_SPEED,tlen=qlen)
acc_st=State(x=0,y=0)
itr=0
#fig,axes=plt.subplots(nrows=2,ncols=1,figsize=(10,10))
fig=plt.figure(figsize=(10,10))
gs = fig.add_gridspec(3, 1)
ax1=fig.add_subplot(gs[0,:])
ax2=fig.add_subplot(gs[1:,:])
axes=[ax1,ax2]


def map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _ofang=-1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)

    _vect[0]+=_ofx
    _vect[1]+=_ofy
    return [_vect[0],_vect[1]]
def inv_map_plot(vect,_ofx,_ofy,_ofang):
    _vect=np.array(vect,dtype=np.float64).copy()
    _vect[0]-=_ofx
    _vect[1]-=_ofy
    _ofang=1*_ofang
    rot_mat=np.array([[math.cos(_ofang),math.sin(_ofang)],[-math.sin(_ofang),math.cos(_ofang)]])
    _vect=np.matmul(rot_mat,_vect)
    return [_vect[0],_vect[1]]
U=np.zeros((NU,T),dtype=np.float64)
iLQR=iLQRController()
init_time=time.time()
while MAX_TIME >= dis_time:
       
        _full_state = np.array([state.x, state.y, state.yaw],dtype=np.float64)
    
        
        _full_state_traj=_Data_Processor.update_trajectory(_traj)
        xref, dref = _Data_Processor.get_ref_trajectory(state,dl,DT,T)
        n_xref=np.zeros((NX,T+1),dtype=np.float64)
        n_xref[0,:]=xref[0,:]
        n_xref[1,:]=xref[1,:]
        n_xref[2,:]=xref[3,:]
        if _end==True:
            break

        _index+=_dind
        
        #oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(xref, Motion_Status, dref, oa, odelta)
        X,U, cost= iLQR.ilqr(_full_state, U, n_xref )
        oa=U[0,1]
        odelta=U[1,1]
        ox=X[0,1]
        oy=X[1,1]
        ov=U[0,0]
        oyaw=X[2,1]
        state_str=' x: '+str('%0.4f'%state.x)+' y: '+str('%0.4f'%state.y)+' v: '+str('%0.4f'%state.v)+' yaw: '+str('%0.4f'%np.rad2deg(state.yaw))+'\n'
        xref_str =' x: '+str('%0.4f'%xref[0][0])+' y: '+str('%0.4f'%xref[1][0])+' v: '+str('%0.4f'%xref[2][0])+' yaw: '+str('%0.4f'%np.rad2deg(xref[3][0]))+'\n'
        time_str ='Time(Sec): '+str('%0.4f'%dis_time)+' Sec Distance(m): '+str('%0.4f'%_index)+' m '+'\n'
        #xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        act_str  =' accel: '+str('%0.4f'%accl)+' gas: '+str('%0.4f'%gas)+' steer: '+str('%0.4f'%np.rad2deg(steer))+'\n'
    
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        max_steer=np.deg2rad(45)
        max_accl=5
        if odelta is not None:
            if abs(U[1,0])<max_steer:
                  di= U[1,0]
            else:
                di=(U[1,0]/abs(U[1,0]))*max_steer
            
            if abs(U[0,0])<max_accl:
                  ai=U[0,0]
            else:
                ai=(U[0,0]/abs(U[0,0]))*max_accl
        accl=ai
        steer=di
        
        #act_str=str(steer)+'\n'
        _log_state=[ofx,ofy,_full_state[0],ofang]
        nst,path,offset,img,rw,ter,info=Env.step([steer,accl,0])
        img_raw=Env.render(mode='rgb_array')
        _traj,_dind,_end=path
        ofx,ofy,ofang=offset
        
        gas=Env.gas
        
        state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        _log_next_state=[ofx,ofy,state.v,ofang]
        #Motion_Status.set_state(next_state)
        curr_time=time.time()-init_time
        log_state.append(_log_state)
        log_traj.append(_full_state_traj)
        log_img.append(img)
        log_act.append([accl,steer,0])
        log_next_state.append(_log_next_state)
        log_reward.append([dis_time,curr_time,_index,rw,ter])
        
        dis_time = dis_time + DT
        if car_move==True:
            _xref=xref.copy()
            [_ox,_oy]=map_plot([ox,oy],ofx,ofy,ofang)
            [_xref[0,:],_xref[1,:]]=map_plot([xref[0,:],xref[1,:]],ofx,ofy,ofang)
            [_x,_y]=map_plot([state.x,state.y],ofx,ofy,ofang)
            car_x,car_y,car_ang=ofx,ofy,ofang
            _mpx,_mpy=mpx,mpy
        else:
            _ox,_oy=ox,oy
            _xref=xref
            _x,_y=state.x,state.y
            car_x,car_y,car_ang=state.x,state.y,state.yaw
            [_mpx,_mpy]=inv_map_plot([mpx,mpy],ofx,ofy,ofang)
        x.append(_x)
        y.append(_y)
        yaw.append(ofang)
        v.append(state.v)
        t.append(dis_time)
        d.append(di)
        a.append(ai)

        
    
      
        
        if show_animation:  # pragma: no cover
            axes[0].cla()
            axes[1].cla()
            axes[0].text(.01,.6,'Running : '+time_str,fontsize=10,color='c')
            axes[0].text(.01,.4,'state : '+state_str,fontsize=10,color='r')
            axes[0].text(.01,.2,'xref : '+xref_str,fontsize=10,color='b')
            axes[0].text(.01,0,'act : '+act_str,fontsize=10,color='g')
            
            if ox is not None:
                axes[1].plot(_ox, _oy, "xr", label="MPC",alpha=1.0)
            axes[1].plot(_mpx, _mpy, "-b", label="course")
            axes[1].plot(x, y, "oc", label="trajectory")
            axes[1].plot(_xref[0, :], _xref[1, :], "xk", label="xref",alpha=1.0)
            axes[1].plot(_mpx[_index], _mpy[_index], "xg", label="target",alpha=1.0)
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            car_points=Draw_Car(car_x,car_y)
            rp=Move_Contour(car_points,0,0,car_ang)
            r_car_points=rp.apply()
            r_points=np.array(r_car_points,dtype=np.float64)
            axes[1].plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            #plt.title("Time[s]:" + str(round(time, 2))
            #          + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
            #          + '\nyaw'+str(state.yaw)
            #          + '\nxref_yaw'+str(xref[3][0])
            #          + '\nsteer'+str(steer))
            #fig.canvas.draw()
            plt.pause(0.0001)
            if rec==True:
                if itr%1==0:
                    plt.savefig('Fig_iLQR/'+str(int(itr/1))+'.jpg')
                    cv2.imwrite('Rec_iLQR/'+str(int(itr/1))+'.jpg',img)
                    cv2.imwrite('Rec_iLQR_Raw/'+str(int(itr/1))+'.jpg',img_raw)
            itr+=1
            if ter==True:
                print('terminal ... ')
                
if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(car_x, car_y, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


return ii 9 509.62541470024746 509.6254147002474
return ii 9 432.90563345350495 432.90563345350495
return ii 9 364.3772415379583 364.3772415379583
return ii 9 304.0402389536077 304.0402389536077
break 1 1
return ii 1 251.89462570045274 251.89462570045276
break 1 1
return ii 1 208.69925753316915 208.6992575331692
break 1 1
return ii 1 342.9114148319518 342.91141483195184
return ii 9 285.36375959527896 285.36375959527896
return ii 9 236.00749368980175 236.00749368980172
return ii 9 378.79344357947025 378.79344357947025
return ii 9 316.6490452340978 316.6490452340978
return ii 9 262.6960362199213 262.6960362199213
return ii 9 216.93441653694111 216.93441653694111
return ii 9 353.93009893145734 353.9300989314572
return ii 9 294.9352297231537 294.9352297231537
return ii 9 244.13174984604612 244.13174984604612
return ii 9 203.03930740869725 203.03930740869725
return ii 9 335.2226146389065 335.2226146389065
return ii 9 278.7052853291919 278.7052853291919
return ii 9 230.37934535067333 230.379

break 1 1
return ii 1 286.21105209172674 286.2110520918264
break 1 1
return ii 1 236.72509991547776 236.7250999159856
break 1 1
return ii 1 379.71310174617275 379.7131017583563
break 1 1
return ii 1 317.45507501555824 317.45507501566067
break 1 1
return ii 1 263.3884375918182 263.38843759192423
break 1 1
return ii 1 217.51318941356058 217.51318941377372
break 1 1
return ii 1 354.6975119686981 354.6975119721777
break 1 1
return ii 1 295.6030844335041 295.60308443351903
break 1 1
return ii 1 244.70004619309114 244.70004619312547
break 1 1
return ii 1 203.45336473440182 203.45336473444848
break 1 1
return ii 1 335.7885145913161 335.7885145918879
break 1 1
return ii 1 279.19475656069017 279.1947565606925
break 1 1
return ii 1 230.79238785733708 230.7923878573402
break 1 1
return ii 1 372.0775169402285 372.07751694027354
break 1 1
return ii 1 310.76889691694254 310.7688969169427
break 1 1
return ii 1 257.65166623688015 257.6516662368808
break 1 1
return ii 1 212.90203665876754 212.902036658

break 1 1
return ii 1 379.7347330682484 379.7347330711352
break 1 1
return ii 1 317.4740341731481 317.4740341731954
break 1 1
return ii 1 263.404725119417 263.404725119517
break 1 1
return ii 1 217.52680569735577 217.52680569761384
break 1 1
return ii 1 354.71555459274566 354.71555459763886
break 1 1
return ii 1 295.6187873655188 295.6187873655411
break 1 1
return ii 1 244.71340972654102 244.71340972670095
break 1 1
return ii 1 203.46310128086782 203.46310128111685
break 1 1
return ii 1 335.80181881573105 335.8018188203282
break 1 1
return ii 1 279.20626492791905 279.2062649279421
break 1 1
return ii 1 230.80210047022197 230.80210047037173
break 1 1
return ii 1 372.0900819801885 372.09008198344463
break 1 1
return ii 1 310.77988834240256 310.7798883424267
break 1 1
return ii 1 257.6610840887585 257.66108408880757
break 1 1
return ii 1 212.90891569577946 212.90891569587757
break 1 1
return ii 1 348.56767835555695 348.56767835720444
break 1 1
return ii 1 290.27292593055876 290.2729259305

break 1 1
return ii 1 365.9887468290761 365.98875403391634
break 1 1
return ii 1 305.4476110014155 305.44761104276756
break 1 1
return ii 1 253.0978804733149 253.0978805589891
break 1 1
return ii 1 209.5783093107295 209.5783094570616
break 1 1
return ii 1 344.09531681399403 344.09531917546497
break 1 1
return ii 1 286.3908113946969 286.39081140374793
break 1 1
return ii 1 236.87767704733318 236.87767707405158
break 1 1
return ii 1 379.9066322936876 379.9066330042955
break 1 1
return ii 1 317.6248175100181 317.6248175141801
break 1 1
return ii 1 263.53438901126935 263.5343890135079
break 1 1
return ii 1 217.63534770622587 217.63534770899133
break 1 1
return ii 1 354.85782271914206 354.8578227481267
break 1 1
return ii 1 295.7426225519616 295.74262255231986
break 1 1
return ii 1 244.8188243491341 244.8188243509636
break 1 1
return ii 1 203.53998835027892 203.53998835339297
break 1 1
return ii 1 335.9059559640041 335.90595603194396
break 1 1
return ii 1 279.2963237200442 279.2963237213192

break 1 1
return ii 1 357.9159623301706 357.9293120706008
break 1 1
return ii 1 297.7934616591061 297.79432222549536
break 1 1
return ii 1 246.3050825768433 246.30583909341624
break 1 1
return ii 1 205.58625960076253 205.58703443958893
break 1 1
return ii 1 339.8571254899207 339.87266898767757
break 1 1
return ii 1 282.5993108654084 282.6000140604374
break 1 1
return ii 1 233.84152948779018 233.842280097076
break 1 1
return ii 1 379.1127247712837 379.12692389364344
break 1 1
return ii 1 316.8538075890615 316.85452595941933
break 1 1
return ii 1 263.1058023343667 263.10620595260457
break 1 1
return ii 1 218.32808510415222 218.3286250301469
break 1 1
return ii 1 358.3307021046586 358.34049120681595
break 1 1
return ii 1 298.95577228556385 298.95602188209193
break 1 1
return ii 1 247.9165969341106 247.91686633693
break 1 1
return ii 1 206.8873249688731 206.88767184657672
break 1 1
return ii 1 341.90832307260246 341.91456854411695
break 1 1
return ii 1 284.75338488976496 284.75347196705343

break 1 1
return ii 1 307.136621944112 307.13662194420044
break 1 1
return ii 1 254.5418210416134 254.54182104234076
break 1 1
return ii 1 210.6312415677661 210.63124156930988
break 1 1
return ii 1 345.5135851912456 345.51358522144733
break 1 1
return ii 1 287.6211055926404 287.62110559279125
break 1 1
return ii 1 237.92001739908372 237.92001740041076
break 1 1
return ii 1 198.5166511625082 198.51665116431298
break 1 1
return ii 1 329.0041673207644 329.0041673559835
break 1 1
return ii 1 273.3331212068423 273.333121207072
break 1 1
return ii 1 225.85346535576596 225.85346535711926
break 1 1
return ii 1 365.66229151355947 365.66229154106304
break 1 1
return ii 1 305.1621261153597 305.1621261155229
break 1 1
return ii 1 252.85335059355072 252.85335059405497
break 1 1
return ii 1 209.39884963207317 209.39884963298277
break 1 1
return ii 1 343.8549848112881 343.85498482673654
break 1 1
return ii 1 286.1820726367117 286.182072636773
break 1 1
return ii 1 236.70054988228907 236.7005498826280

break 1 1
return ii 1 345.6893263445516 345.69313513999793
break 1 1
return ii 1 287.4882741407296 287.48841980009
break 1 1
return ii 1 237.65949079727983 237.65972978274837
break 1 1
return ii 1 198.55285363719997 198.5531292432787
break 1 1
return ii 1 328.5708763663206 328.5761663788789
break 1 1
return ii 1 272.7993593894102 272.7995475587625
break 1 1
return ii 1 225.37609415285095 225.3764219970908
break 1 1
return ii 1 363.1099430687492 363.115400260251
break 1 1
return ii 1 302.6127687374528 302.61305028862444
break 1 1
return ii 1 250.53557265723623 250.53586654605903
break 1 1
return ii 1 208.146809808494 208.14719227790945
break 1 1
return ii 1 340.4343734941681 340.44122729141066
break 1 1
return ii 1 282.92767080322864 282.9280151271362
break 1 1
return ii 1 233.8125781304643 233.81303474919878
break 1 1
return ii 1 195.9368264989577 195.93729020185958
break 1 1
return ii 1 325.6586548132144 325.66817314152854
break 1 1
return ii 1 270.2928735868077 270.2932896183029
brea

break 1 1
return ii 1 306.94049424955915 306.9404943435205
break 1 1
return ii 1 254.38020618114572 254.38020623960267
break 1 1
return ii 1 210.5269308638104 210.52693092442905
break 1 1
return ii 1 345.2826566523803 345.28265670439
break 1 1
return ii 1 287.4209504488306 287.42095046392456
break 1 1
return ii 1 237.75125241670588 237.75125253228663
break 1 1
return ii 1 198.39771747122936 198.3977176374165
break 1 1
return ii 1 328.80286295496364 328.80286731461456
break 1 1
return ii 1 273.15865822178836 273.1586582799325
break 1 1
return ii 1 225.7064016070333 225.70640195694483
break 1 1
return ii 1 365.45958221465503 365.4595894062661
break 1 1
return ii 1 304.9844663538895 304.9844663999529
break 1 1
return ii 1 252.70119866091056 252.70119886623803
break 1 1
return ii 1 209.2884660260084 209.28846641289368
break 1 1
return ii 1 343.70533218824653 343.70533921813626
break 1 1
return ii 1 286.05217903560526 286.05217906623477
break 1 1
return ii 1 236.5905963069467 236.5905965200

In [10]:
log_dict={
'state':log_state,
'traj':log_traj,
'image':log_img,
'action':log_act,
'next_sate':log_next_state,
'time_reward':log_reward
}

In [11]:
import pickle
f=open('dataset_iLQR','wb')
pickle.dump(log_dict,f)
f.close()
itr

1034

In [12]:
f=open('dataset_iLQR','rb')
dataset=pickle.load(f)
f.close()

In [13]:
dataset.keys()

dict_keys(['state', 'traj', 'image', 'action', 'next_sate', 'time_reward'])

In [14]:
traj=np.array(log_traj)#np.array(dataset['traj'])
plt.plot(traj[])
plt.show()

SyntaxError: invalid syntax (<ipython-input-14-b5367520e56e>, line 2)

In [ ]:
traj[200]